In [1]:
# Importing files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import sys
import glob
import datetime
import calendar
import csv

# Spark imports
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import col, asc, desc, split

from pyspark.sql.functions import array_contains # used to check if the array has a value

In [2]:
# Initialize a spark session.
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [3]:
spark = init_spark() # Initializate spark

In [4]:
# BUSINESS.JSON

In [5]:
# Importing Business.json file
path = 'data/yelp_academic_dataset_business.json'
df_business = spark.read.json(path)

In [6]:
# Print the total number of rows and columns
print((df_business.count(), len(df_business.columns)))

(150346, 14)


In [7]:
# Print the columns
df_business.columns

['address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'name',
 'postal_code',
 'review_count',
 'stars',
 'state']

In [8]:
# Print the dtypes of each column
df_business.dtypes

[('address', 'string'),
 ('attributes',
  'struct<AcceptsInsurance:string,AgesAllowed:string,Alcohol:string,Ambience:string,BYOB:string,BYOBCorkage:string,BestNights:string,BikeParking:string,BusinessAcceptsBitcoin:string,BusinessAcceptsCreditCards:string,BusinessParking:string,ByAppointmentOnly:string,Caters:string,CoatCheck:string,Corkage:string,DietaryRestrictions:string,DogsAllowed:string,DriveThru:string,GoodForDancing:string,GoodForKids:string,GoodForMeal:string,HairSpecializesIn:string,HappyHour:string,HasTV:string,Music:string,NoiseLevel:string,Open24Hours:string,OutdoorSeating:string,RestaurantsAttire:string,RestaurantsCounterService:string,RestaurantsDelivery:string,RestaurantsGoodForGroups:string,RestaurantsPriceRange2:string,RestaurantsReservations:string,RestaurantsTableService:string,RestaurantsTakeOut:string,Smoking:string,WheelchairAccessible:string,WiFi:string>'),
 ('business_id', 'string'),
 ('categories', 'string'),
 ('city', 'string'),
 ('hours',
  'struct<Friday:st

In [9]:
# Printing how many Null files each categorie have
for i in df_business.columns:
    print(i, df_business.filter(col(i).isNull()).count())

address 0
attributes 13744
business_id 0
categories 103
city 0
hours 23223
is_open 0
latitude 0
longitude 0
name 0
postal_code 0
review_count 0
stars 0
state 0


In [10]:
# Printing the categories column
df_business.select("categories").show(25, False) # Printing the categories in the dataset for the first 25 instances

# ----------------- Comments --------------
# 1 - Each row is given as a single string

+----------------------------------------------------------------------------------------------------------+
|categories                                                                                                |
+----------------------------------------------------------------------------------------------------------+
|Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists|
|Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services                            |
|Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores                        |
|Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries                                                     |
|Brewpubs, Breweries, Food                                                                                 |
|Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants                              |
|Sporting Goods, Fa

In [11]:
# Printing the categories in the dataset for the first 25 instances
df_business.select("attributes").show(10, False) 

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|attributes                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
+---------------

In [12]:
# Filtering the rows if the categories contains RESTAURANT
df_business_filtered = df_business.filter(df_business.categories.contains('Restaurants'))

In [13]:
# Print the number of rows and columns of the dataframe filtered
print((df_business_filtered.count(), len(df_business_filtered.columns)))

(52268, 14)


In [14]:
# Printing how many Null files each categorie have after filtering
for i in df_business.columns:
    print(i, df_business_filtered.filter(col(i).isNull()).count())

address 0
attributes 565
business_id 0
categories 0
city 0
hours 7278
is_open 0
latitude 0
longitude 0
name 0
postal_code 0
review_count 0
stars 0
state 0


In [15]:
# Apllying one-hot encoding to categories feature

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

df = df_business_filtered.toPandas()
mlb = MultiLabelBinarizer()
df['categories'] = df['categories'].str.split(',')

df_categories = pd.DataFrame(mlb.fit_transform(df['categories']),columns=mlb.classes_, index=df.index)
df_categories

,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,Adult,Adult Entertainment,Advertising,Afghan,African,...,Wholesale Stores,Wholesalers,Wigs,Wine Bars,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yoga
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Counting the categories
categories_counts = df['categories'].value_counts()
categories_counts

[Restaurants,  Pizza]                                                                                                                                                                                                                935
[Pizza,  Restaurants]                                                                                                                                                                                                                823
[Restaurants,  Mexican]                                                                                                                                                                                                              728
[Restaurants,  Chinese]                                                                                                                                                                                                              708
[Mexican,  Restaurants]                                             

In [17]:
# Pre-processing the feature City
# Changing all letter to lowercase

df = df.sort_values(by=['city'])  # Sorting by name
print('Original unique city values:', len(df['city'].unique()))  # Printing the unique values
df['city'] = df['city'].str.lower()
print('Unique city values after changing all letter to lowercase:', len(df['city'].unique())) # Printing the unique values

Original unique city values: 920
Unique city values after changing all letter to lowercase: 859


In [18]:
# Encoding feature City and State features

df_city = pd.get_dummies(df.city, prefix='City')
df_state = pd.get_dummies(df.state, prefix='State')
df_state # printing for state (example)

,State_AB,State_AZ,State_CA,State_CO,State_DE,State_FL,State_HI,State_ID,State_IL,State_IN,State_LA,State_MO,State_MT,State_NC,State_NJ,State_NV,State_PA,State_TN,State_XMS
25839,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
42911,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
33631,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
6695,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9078,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3171,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
48502,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
25977,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
43985,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4260,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [19]:
# # Counting the categories
categories_counts = df['city'].value_counts()
categories_counts

philadelphia               5854
tampa                      2967
indianapolis               2862
nashville                  2504
tucson                     2469
new orleans                2261
edmonton                   2168
saint louis                1790
reno                       1288
boise                       848
santa barbara               767
clearwater                  679
wilmington                  620
st. louis                   544
metairie                    522
saint petersburg            491
franklin                    444
st. petersburg              405
sparks                      334
brandon                     327
largo                       312
meridian                    312
cherry hill                 297
carmel                      291
west chester                281
kenner                      228
new port richey             227
goleta                      220
palm harbor                 208
greenwood                   207
                           ... 
bywater 

In [20]:
###############
## USER FILE ##
###############

In [21]:
# Loading the user file
path = 'data/yelp_academic_dataset_user.json'
df_user = spark.read.json(path)

In [22]:
# Print the number of rows and columns
print((df_user.count(), len(df_user.columns)))

(1987897, 22)


In [23]:
#Printing the columns
df_user.columns

['average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'elite',
 'fans',
 'friends',
 'funny',
 'name',
 'review_count',
 'useful',
 'user_id',
 'yelping_since']

In [24]:
# Printing how many Null files each categorie have
for i in df_user.columns:
    print(i, df_user.filter(col(i).isNull()).count())

average_stars 0
compliment_cool 0
compliment_cute 0
compliment_funny 0
compliment_hot 0
compliment_list 0
compliment_more 0
compliment_note 0
compliment_photos 0
compliment_plain 0
compliment_profile 0
compliment_writer 0
cool 0
elite 0
fans 0
friends 0
funny 0
name 0
review_count 0
useful 0
user_id 0
yelping_since 0


In [35]:
df_user_filtered = df_user.select('user_id', 'review_count')

In [38]:
df_user_filtered.show(10)

+--------------------+------------+
|             user_id|review_count|
+--------------------+------------+
|qVc8ODYU5SZjKXVBg...|         585|
|j14WgRoU_-2ZE1aw1...|        4333|
|2WnXYQFK0hXEoTxPt...|         665|
|SZDeASXq7o05mMNLs...|         224|
|hA5lMy-EnncsH4JoR...|          79|
|q_QQ5kBBwlCcbL1s4...|        1221|
|cxuxXkcihfCbqt5By...|          12|
|E9kcWJdJUHuTKfQur...|         358|
|lO1iq-f75hnPNZkTy...|          40|
|AUi8MPWJ0mLkMfwbu...|         109|
+--------------------+------------+
only showing top 10 rows



In [40]:
df_user_filtered = df_user_filtered.filter(col('review_count')>=10)

In [42]:
# Printing the number of users with more than 10 reviews
df_user_filtered.count()

726519

In [25]:
#################
## REVIEW FILE ##
#################

In [26]:
path = 'data/yelp_academic_dataset_review.json'
df_review = spark.read.json(path)

In [27]:
# Print the number of rows and columns
print((df_review.count(), len(df_review.columns)))

(6990280, 9)


In [28]:
#Printing the columns
df_review.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [29]:
# Printing how many Null files each categorie have
for i in df_review.columns:
    print(i, df_review.filter(col(i).isNull()).count())

business_id 0
cool 0
date 0
funny 0
review_id 0
stars 0
text 0
useful 0
user_id 0


In [62]:
df_review_filtered = df_review.select('business_id', 'review_id', 'stars','user_id')
df_review_filtered.show(20)

+--------------------+--------------------+-----+--------------------+
|         business_id|           review_id|stars|             user_id|
+--------------------+--------------------+-----+--------------------+
|XQfwVwDr-v0ZS3_Cb...|KU_O5udG6zpxOg-Vc...|  3.0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|BiTunyQ73aT9WBnpR...|  5.0|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|saUsX_uimxRlCVr67...|  3.0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|AqPFMleE6RsU23_au...|  5.0|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|Sx8TMOWLNuJBWer-0...|  4.0|bcjbaE6dDog4jkNY9...|
|04UD14gamNjLY0IDY...|JrIxlS1TzJ-iCu79u...|  1.0|eUta8W_HdHMXPzLBB...|
|gmjsEdUsKpj9Xxu6p...|6AxgBCNX_PNTOxmbR...|  5.0|r3zeYsv1XFBRA4dJp...|
|LHSTtnW3YHCeUkRDG...|_ZeMknuYdlQcUqng_...|  5.0|yfFzsLmaWF2d4Sr0U...|
|B5XSoSG3SfvQGtKEG...|ZKvDG2sBvHVdF5oBN...|  3.0|wSTuiTk-sKNdcFypr...|
|gebiRewfieSdtt17P...|pUycOfUwM8vqX7KjR...|  3.0|59MxRhNVhU9MYndMk...|
|uMvVYRgGNXf5boolA...|rGQRf8UafX7OTlMNN...|  5.0|1WHRWwQmZOZDAhp2Q...|
|EQ-TZ

In [66]:
# Filtering REVIEWS by only users identified with >= 20 REVIEWS
df_review_filtered2 = df_review_filtered.join(df_user_filtered, df_review_filtered.user_id == df_user_filtered.user_id ,"left_semi")
df_review_filtered2.count()

4863844

In [71]:
# Filtering REVIEWS file by only RESTAURANTS identified with the restaurant CATEGORIES
df_review_filtered3 = df_review_filtered2.join(df_business_filtered[['business_id']], df_review_filtered2.business_id == df_business_filtered.business_id ,"left_semi")
df_review_filtered3.count()

3432479

In [106]:
df_review_filtered4 = df_review_filtered3[['user_id','business_id']].distinct()

In [107]:
df_review_filtered4.count()

3290578

In [100]:
i1 = np.array([[1, 8, 6],
      [-2, 5, 9],
      [1, 1, 1]])

u = np.array([1, 3, 5])

np.matmul(u,i1)/np.sum(u)

array([0.        , 3.11111111, 4.22222222])